In [1]:
import duckdb
import pandas as pd

# Open an in-memory DuckDB connection
con = duckdb.connect()

# Load the merged CSVs into DuckDB tables for faster repeated queries
con.execute("""
CREATE OR REPLACE TABLE last12 AS
SELECT * 
FROM read_csv_auto('../../data/processed/master_last12.csv', HEADER=TRUE);
""")
con.execute("""
CREATE OR REPLACE TABLE longterm AS
SELECT * 
FROM read_csv_auto('../../data/processed/master_longterm.csv', HEADER=TRUE);
""")

# Verify the tables exist
print(con.execute("SHOW TABLES;").df())


       name
0    last12
1  longterm


In [2]:
last12_preview = con.execute("""
  SELECT * 
  FROM last12 
  ORDER BY started_at 
  LIMIT 5;
""").df()
last12_preview

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,38C83025453F5D63,electric_bike,2024-05-01 00:00:41,2024-05-01 00:10:32,None,None,None,None,41.950000,-87.730000,41.920000,-87.720000,casual
1,5D5A221AED91BE56,electric_bike,2024-05-01 00:01:42,2024-05-01 00:09:21,Southport Ave & Clark St,TA1308000047,None,None,41.957161,-87.664227,41.950000,-87.670000,member
2,86A78279AD258550,electric_bike,2024-05-01 00:01:43,2024-05-01 00:07:51,None,None,None,None,42.050000,-87.670000,42.060000,-87.670000,casual
3,E92D44EC0F271F6D,electric_bike,2024-05-01 00:01:58,2024-05-01 00:05:45,Dearborn Pkwy & Delaware Pl,TA1307000128,Clark St & Schiller St,TA1309000024,41.899015,-87.629916,41.907993,-87.631501,member
4,C3C5ED7E56D1D893,electric_bike,2024-05-01 00:02:31,2024-05-01 00:25:52,None,None,None,None,41.920000,-87.630000,41.940000,-87.640000,casual


In [3]:
longterm_preview = con.execute("""
  SELECT * 
  FROM longterm 
  ORDER BY started_at 
  LIMIT 5;
""").df()
longterm_preview

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,A3F8D895163BBB49,electric_bike,2021-01-01 00:02:05,2021-01-01 00:12:39,None,None,None,None,41.980000,-87.650000,41.980000,-87.660000,member
1,0D139A3203274B87,classic_bike,2021-01-01 00:02:24,2021-01-01 00:08:39,State St & 33rd St,13216,MLK Jr Dr & 29th St,TA1307000139,41.834734,-87.625813,41.842052,-87.617000,member
2,C7AE8E9CDB197A8E,classic_bike,2021-01-01 00:06:55,2021-01-01 00:26:36,Lakeview Ave & Fullerton Pkwy,TA1309000019,Ritchie Ct & Banks St,KA1504000134,41.925858,-87.638973,41.906866,-87.626217,member
3,2633EB2B8A99F5CB,electric_bike,2021-01-01 00:12:13,2021-01-01 00:20:06,Kedzie Ave & Milwaukee Ave,13085,None,None,41.929528,-87.707899,41.920000,-87.720000,member
4,3097EF26414C7016,classic_bike,2021-01-01 00:12:21,2021-01-01 00:12:33,Montrose Harbor,TA1308000012,Montrose Harbor,TA1308000012,41.963982,-87.638181,41.963982,-87.638181,member


# 📊 Credibility Assessment

In this step, we verify the integrity of the **last12** and **longterm** datasets by checking for:

- **Null values** in critical fields (`ride_id`, `rideable_type`, `member_casual`, `start_station_name`, `end_station_name`)
- **Duplicate** `ride_id` entries

The following SQL queries will be executed in DuckDB to surface any issues before moving on to cleaning and filtering.  

In [4]:
# Null counts in critical fields

df_nulls_last12 = con.execute("""
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN ride_id             IS NULL THEN 1 ELSE 0 END) AS null_ride_id,
  SUM(CASE WHEN rideable_type       IS NULL THEN 1 ELSE 0 END) AS null_rideable_type,
  SUM(CASE WHEN member_casual       IS NULL THEN 1 ELSE 0 END) AS null_member_casual,
  SUM(CASE WHEN start_station_name  IS NULL THEN 1 ELSE 0 END) AS null_start_station_name,
  SUM(CASE WHEN end_station_name    IS NULL THEN 1 ELSE 0 END) AS null_end_station_name
FROM last12;
""").df()
print("Merged last12 Data Null Counts:")
print(df_nulls_last12.to_string(index=False))

Merged last12 Data Null Counts:
 total_rows  null_ride_id  null_rideable_type  null_member_casual  null_start_station_name  null_end_station_name
    6238340           0.0                 0.0                 0.0                1190993.0              1223471.0


In [5]:
df_nulls_longterm = con.execute("""
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN ride_id             IS NULL THEN 1 ELSE 0 END) AS null_ride_id,
  SUM(CASE WHEN rideable_type       IS NULL THEN 1 ELSE 0 END) AS null_rideable_type,
  SUM(CASE WHEN member_casual       IS NULL THEN 1 ELSE 0 END) AS null_member_casual,
  SUM(CASE WHEN start_station_name  IS NULL THEN 1 ELSE 0 END) AS null_start_station_name,
  SUM(CASE WHEN end_station_name    IS NULL THEN 1 ELSE 0 END) AS null_end_station_name
FROM longterm;
""").df()
print("Merged longterm Data Null Counts:")
print(df_nulls_longterm.to_string(index=False))

Merged longterm Data Null Counts:
 total_rows  null_ride_id  null_rideable_type  null_member_casual  null_start_station_name  null_end_station_name
   16982657           0.0                 0.0                 0.0                2399589.0              2561114.0


In [6]:
# Duplicated ride_id counts
df_dupes_last12 = con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT ride_id)                  AS unique_ride_ids,
  (COUNT(*) - COUNT(DISTINCT ride_id))     AS duplicate_ride_id_count
FROM last12;
""").df()
print("\nMerged last12 Data Duplicate Ride IDs:")
print(df_dupes_last12.to_string(index=False))


Merged last12 Data Duplicate Ride IDs:
 total_rows  unique_ride_ids  duplicate_ride_id_count
    6238340          6238129                      211


In [7]:
# Duplicated ride_id counts
df_dupes_longterm = con.execute("""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT ride_id)                  AS unique_ride_ids,
  (COUNT(*) - COUNT(DISTINCT ride_id))     AS duplicate_ride_id_count
FROM longterm;
""").df()
print("\nMerged longterm Data Duplicate Ride IDs:")
print(df_dupes_longterm.to_string(index=False))


Merged longterm Data Duplicate Ride IDs:
 total_rows  unique_ride_ids  duplicate_ride_id_count
   16982657         16982657                        0


## 📝 Additional Credibility Checks

Before moving to data cleaning, we performed extra validation:

- Checked that all `ride_id` fields are exactly 16 characters.
- Verified that `member_casual` contains only two categories: `member` or `casual`.
- Verified that `rideable_type` contains only correct categories like `classic_bike` or `electric_bike`.

Below are the SQL queries for these checks:

In [8]:
# Ride_id length check
bad_len_last12 = con.execute("""
SELECT COUNT(*) AS bad_length_ride_id
FROM last12
WHERE LENGTH(ride_id) != 16;
""").fetchone()[0]
print(f"Rows with ride_id not 16 chars (last12): {bad_len_last12}")

bad_len_longterm = con.execute("""
SELECT COUNT(*) AS bad_length_ride_id
FROM longterm
WHERE LENGTH(ride_id) != 16;
""").fetchone()[0]
print(f"Rows with ride_id not 16 chars (longterm): {bad_len_longterm}")

# Unexpected values in member_casual
bad_mem_last12 = con.execute("""
SELECT COUNT(*) FROM last12
WHERE member_casual NOT IN ('member', 'casual');
""").fetchone()[0]
print(f"Rows with unexpected member_casual (last12): {bad_mem_last12}")

bad_mem_longterm = con.execute("""
SELECT COUNT(*) FROM longterm
WHERE member_casual NOT IN ('member', 'casual');
""").fetchone()[0]
print(f"Rows with unexpected member_casual (longterm): {bad_mem_longterm}")

# Unique rideable_type values in last12
types_last12 = con.execute("""
SELECT DISTINCT rideable_type
FROM last12;
""").fetchdf()
print("Unique rideable_type values in last12:\n", types_last12)

# Unique rideable_type values in longterm
types_longterm = con.execute("""
SELECT DISTINCT rideable_type
FROM longterm;
""").fetchdf()
print("Unique rideable_type values in longterm:\n", types_longterm)

Rows with ride_id not 16 chars (last12): 0
Rows with ride_id not 16 chars (longterm): 0
Rows with unexpected member_casual (last12): 0
Rows with unexpected member_casual (longterm): 0
Unique rideable_type values in last12:
       rideable_type
0  electric_scooter
1      classic_bike
2     electric_bike
Unique rideable_type values in longterm:
    rideable_type
0  electric_bike
1   classic_bike
2    docked_bike


## 📝 Credibility Assessment: Key Findings & Next Steps

- **No nulls** in `ride_id`, `rideable_type`, or `member_casual` in either dataset.
- **Significant nulls** in `start_station_name` and `end_station_name` (≈19%–21% of rows). These fields may require imputation or exclusion depending on downstream analysis needs.
- **Duplicates detected**:  
  - `last12`: 211 duplicate `ride_id` rows (will be dropped)
  - `longterm`: no duplicates

**Next steps:**  
- Drop all rows with duplicate `ride_id` in `last12`
- Drop or flag rows with missing station names
- Save new cleaned tables: `cleaned_last12.csv` and `cleaned_longterm.csv`

In [9]:
# Keep only unique ride_id and delete rows with missing 
# start_station_name or end_station_name for last12 dataset
con.execute("""
CREATE OR REPLACE TABLE last12_cleaned AS
SELECT *
FROM (
  SELECT DISTINCT ON (ride_id) *
  FROM last12
  WHERE start_station_name IS NOT NULL
    AND end_station_name   IS NOT NULL
  ORDER BY ride_id, started_at
)
ORDER BY started_at;
""")

In [10]:
# Load new variable df_last12_cleaned 
df_last12_cleaned = con.execute("SELECT * FROM last12_cleaned;").df()
# last12 preview shape and sorting check
print(f"last12_cleaned shape: {df_last12_cleaned.shape}")
df_last12_cleaned.head()

last12_cleaned shape: (4401707, 13)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E92D44EC0F271F6D,electric_bike,2024-05-01 00:01:58,2024-05-01 00:05:45,Dearborn Pkwy & Delaware Pl,TA1307000128,Clark St & Schiller St,TA1309000024,41.899015,-87.629916,41.907993,-87.631501,member
1,55FA739D575E7AE3,classic_bike,2024-05-01 00:02:35,2024-05-01 00:05:54,Dearborn Pkwy & Delaware Pl,TA1307000128,Clark St & Schiller St,TA1309000024,41.898969,-87.629912,41.907993,-87.631501,member
2,7A296E31619D21E9,classic_bike,2024-05-01 00:04:31,2024-05-01 00:20:13,Broadway & Barry Ave,13137,Broadway & Wilson Ave,13074,41.937582,-87.644098,41.965221,-87.658139,member
3,2FB602B952B83DB7,electric_bike,2024-05-01 00:04:34,2024-05-01 00:12:17,Broadway & Granville Ave,15571,Sheridan Rd & Loyola Ave,RP-009,41.994770,-87.660287,42.001044,-87.661198,casual
4,2D37389234DB639E,electric_bike,2024-05-01 00:05:13,2024-05-01 00:05:48,Western Ave & Ardmore Ave,464,Western Ave & Ardmore Ave,464,41.986607,-87.689669,41.986764,-87.689880,casual


In [11]:
# Delete rows with missing start_station_name or end_station_name for longterm dataset
con.execute("""
CREATE OR REPLACE TABLE longterm_cleaned AS
SELECT *
FROM longterm
WHERE start_station_name IS NOT NULL
  AND end_station_name   IS NOT NULL
ORDER BY started_at;
""")

In [12]:
# Load new variable df_longterm_cleaned 
df_longterm_cleaned = con.execute("SELECT * FROM longterm_cleaned;").df()
# longterm preview shape and sorting check
print(f"longterm_cleaned shape: {df_longterm_cleaned.shape}")
df_longterm_cleaned.head()

longterm_cleaned shape: (13289731, 13)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0D139A3203274B87,classic_bike,2021-01-01 00:02:24,2021-01-01 00:08:39,State St & 33rd St,13216,MLK Jr Dr & 29th St,TA1307000139,41.834734,-87.625813,41.842052,-87.617000,member
1,C7AE8E9CDB197A8E,classic_bike,2021-01-01 00:06:55,2021-01-01 00:26:36,Lakeview Ave & Fullerton Pkwy,TA1309000019,Ritchie Ct & Banks St,KA1504000134,41.925858,-87.638973,41.906866,-87.626217,member
2,3097EF26414C7016,classic_bike,2021-01-01 00:12:21,2021-01-01 00:12:33,Montrose Harbor,TA1308000012,Montrose Harbor,TA1308000012,41.963982,-87.638181,41.963982,-87.638181,member
3,938D5D1998A5470E,classic_bike,2021-01-01 00:12:27,2021-01-01 00:12:30,Montrose Harbor,TA1308000012,Montrose Harbor,TA1308000012,41.963982,-87.638181,41.963982,-87.638181,casual
4,6604F61AE4B14BC1,electric_bike,2021-01-01 00:12:49,2021-01-01 00:43:59,Western Ave & Howard St,527,Campbell Ave & Fullerton Ave,15648,42.018858,-87.690022,41.924680,-87.689328,member


In [13]:
# Save cleaned datasets to CSV
df_last12_cleaned.to_csv('../../data/processed/cleaned_last12.csv', index=False)
df_longterm_cleaned.to_csv('../../data/processed/cleaned_longterm.csv', index=False)

print("✅ Cleaned datasets saved to:")
print("- ../../data/processed/cleaned_last12.csv")
print("- ../../data/processed/cleaned_longterm.csv")

✅ Cleaned datasets saved to:
- ../../data/processed/cleaned_last12.csv
- ../../data/processed/cleaned_longterm.csv
